In [39]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd



# Load the data
df_terminations = pd.read_excel('/Users/timothybrown/Desktop/Terminations.xlsx')
df_active = pd.read_excel('/Users/timothybrown/Desktop/Active.xlsx')

# Convert date columns to datetime format
df_terminations['Term Date'] = pd.to_datetime(df_terminations['Term Date'])
df_active['Adj Service Dt'] = pd.to_datetime(df_active['Adj Service Dt'])

# Create a new column for termination year and hire year
df_terminations['Term Year'] = df_terminations['Term Date'].dt.year
df_active['Hire Year'] = df_active['Adj Service Dt'].dt.year
# Calculate active employees at the end of 2023
end_2023 = "2023-12-31"
active_end_2023 = df_active[df_active['Adj Service Dt'] <= end_2023]
active_end_2023 = active_end_2023[~active_end_2023['Pay Status'].isin(df_terminations['Pay Status'])]
# Initialize an empty dictionary to store yearly data
yearly_summary = {}

# Calculate attrition rate for each year
for year in range(2013, 2024):  # adjust the years as per your requirements
    terminations_in_year = len(df_terminations[df_terminations['Term Year'] == year])
    active_employees_in_year = len(df_active[df_active['Hire Year'] <= year])
    attrition_rate = terminations_in_year / active_employees_in_year * 100  # % rate

    # Store yearly data in the dictionary
    yearly_summary[year] = {"Terminations": terminations_in_year,
                            "Active Employees": active_employees_in_year,
                            "Attrition Rate (%)": attrition_rate}

# Convert dictionary to DataFrame
df_summary = pd.DataFrame.from_dict(yearly_summary, orient='index')

# Print DataFrame
print(df_summary)

# Fit a linear regression model
X = np.array(range(2013, 2024)).reshape(-1, 1)  # Independent variable (years)
y = df_summary['Attrition Rate (%)'].values.reshape(-1, 1)  # Dependent variable (attrition rate)

model = LinearRegression()
model.fit(X, y)

# Forecast the next 5 years
X_forecast = np.array(range(2024, 2029)).reshape(-1, 1)  # Years to forecast
y_forecast = model.predict(X_forecast)

# Calculate potential employees lost based on forecasted attrition rates
active_employees_in_2023 = yearly_summary[2023]['Active Employees']
potential_employees_lost = y_forecast / 100 * active_employees_in_2023

print("Using Linear Regression attrition rate and potential employees lost forecast for the next 5 years:")
for year, attrition_rate, employees_lost in zip(range(2024, 2029), y_forecast, potential_employees_lost):
    print(f"Year {year}: Attrition Rate - {attrition_rate[0]:.2f}%, Potential Employees Lost - {int(employees_lost[0])}")
import matplotlib.pyplot as plt


# write a python code out of current active that can gather the percentage of people that have 5, 10, and 10+ years of service



      Terminations  Active Employees  Attrition Rate (%)
2013             8               169            4.733728
2014            17               187            9.090909
2015             5               213            2.347418
2016            15               221            6.787330
2017             9               236            3.813559
2018            13               265            4.905660
2019            10               293            3.412969
2020            11               305            3.606557
2021            25               328            7.621951
2022            28               339            8.259587
2023            14               343            4.081633
Using Linear Regression attrition rate and potential employees lost forecast for the next 5 years:
Year 2024: Attrition Rate - 5.47%, Potential Employees Lost - 18
Year 2025: Attrition Rate - 5.49%, Potential Employees Lost - 18
Year 2026: Attrition Rate - 5.51%, Potential Employees Lost - 18
Year 2027: Attrition R

In [10]:
# Support Vector Regression
from sklearn.svm import SVR
svr_model = SVR(kernel='rbf')
svr_model.fit(X, np.ravel(y))  # np.ravel() is used to flatten the array
svr_forecast = svr_model.predict(X_forecast)

print("Support Vector Regression forecast for the next 5 years:")
for year, attrition_rate in zip(range(2024, 2029), svr_forecast):
    print(f"Year {year}: {attrition_rate:.2f}%, Potential Employees Lost - {int(attrition_rate / 100 * active_employees_in_2023)}")






Support Vector Regression forecast for the next 5 years:
Year 2024: 4.18%, Potential Employees Lost - 14
Year 2025: 4.10%, Potential Employees Lost - 14
Year 2026: 4.11%, Potential Employees Lost - 14
Year 2027: 4.17%, Potential Employees Lost - 14
Year 2028: 4.22%, Potential Employees Lost - 14


In [35]:

from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
# Polynomial Regression
poly = PolynomialFeatures(degree = 3) 
X_poly = poly.fit_transform(X)
X_poly_forecast = poly.transform(X_forecast)

poly_model = LinearRegression()
poly_model.fit(X_poly, y)
poly_forecast = poly_model.predict(X_poly_forecast)

print("Polynomial Regression forecast for the next 5 years:")
for year, attrition_rate in zip(range(2024, 2029), poly_forecast):
    print(f"Year {year}: {attrition_rate[0]:.2f}%, Potential Employees Lost - {int(attrition_rate[0] / 100 * active_employees_in_2023)}")
    


# Store each model's forecast in a dictionary
forecast_dict = {
    "Linear Regression": y_forecast,
    "Polynomial Regression": poly_forecast,
    "Support Vector Regression": svr_forecast.reshape(-1, 1)  # This reshape transforms SVR output into 2D array
}

# Initialize empty lists to store the averages
avg_attrition_rate = []
avg_employees_lost = []

# Calculate the averages
for year in range(2024, 2029):
    attrition_rate_sum = 0
    employees_lost_sum = 0
    for model in forecast_dict.values():
        attrition_rate = model[year - 2024][0]
        employees_lost = attrition_rate / 100 * active_employees_in_2023
        attrition_rate_sum += attrition_rate
        employees_lost_sum += employees_lost
    avg_attrition_rate.append(attrition_rate_sum / len(forecast_dict))
    avg_employees_lost.append(int(employees_lost_sum / len(forecast_dict)))

# Print the averages
print("Averages based on all regression models:")
for year, attrition_rate, employees_lost in zip(range(2024, 2029), avg_attrition_rate, avg_employees_lost):
    print(f"Year {year}: Average Attrition Rate - {attrition_rate:.2f}%, Average Potential Employees Lost - {employees_lost}")




average_age_at_termination = df_terminations['Age'].mean()
median_age_at_termination = df_terminations['Age'].median()

print(f"Average age at termination: {average_age_at_termination}")
print(f"Median age at termination: {median_age_at_termination}")


Polynomial Regression forecast for the next 5 years:
Year 2024: 5.88%, Potential Employees Lost - 20
Year 2025: 5.56%, Potential Employees Lost - 19
Year 2026: 4.93%, Potential Employees Lost - 16
Year 2027: 3.93%, Potential Employees Lost - 13
Year 2028: 2.51%, Potential Employees Lost - 8
Averages based on all regression models:
Year 2024: Average Attrition Rate - 5.17%, Average Potential Employees Lost - 17
Year 2025: Average Attrition Rate - 5.05%, Average Potential Employees Lost - 17
Year 2026: Average Attrition Rate - 4.85%, Average Potential Employees Lost - 16
Year 2027: Average Attrition Rate - 4.54%, Average Potential Employees Lost - 15
Year 2028: Average Attrition Rate - 4.09%, Average Potential Employees Lost - 14
Average age at termination: 48.58064516129032
Median age at termination: 50.0


In [32]:
import pandas as pd

# Load the datasets
df_active = pd.read_excel('/Users/timothybrown/Desktop/Active.xlsx')
df_terminations = pd.read_excel('/Users/timothybrown/Desktop/Terminations.xlsx')

# Convert date columns to datetime format
df_terminations['Term Date'] = pd.to_datetime(df_terminations['Term Date'])
df_active['Adj Service Dt'] = pd.to_datetime(df_active['Adj Service Dt'])

# Initialize an empty dictionary to store yearly data
yearly_summary = {}

# Calculate attrition rate for each year
for year in range(2013, 2024):  # adjust the years as per your requirements
    terminations_in_year = len(df_terminations[(df_terminations['Term Date'].dt.year == year) & 
                                               (df_terminations['Term Date'] <= pd.Timestamp(year=year, month=12, day=31))])
    
    terminations_after_year = len(df_terminations[(df_terminations['Term Date'].dt.year > year) &
                                                  (df_terminations['Hire Date'] <= pd.Timestamp(year=year, month=12, day=31))])
    
    active_employees_start_year = len(df_active[df_active['Adj Service Dt'] <= pd.Timestamp(year=year, month=1, day=1)])
    hires_in_year = len(df_active[(df_active['Adj Service Dt'].dt.year == year) & 
                                   (df_active['Adj Service Dt'] <= pd.Timestamp(year=year, month=12, day=31))])
    
    active_employees_end_year = active_employees_start_year + hires_in_year - terminations_in_year

    total_workers_in_year = active_employees_start_year + terminations_in_year + terminations_after_year
    attrition_rate = terminations_in_year / total_workers_in_year * 100  # % rate

    # Store yearly data in the dictionary
    yearly_summary[year] = {"Terminations in Year": terminations_in_year,
                            "Total Workers in Year": total_workers_in_year,
                            "Attrition Rate (%)": attrition_rate,
                            "Active Employees Start of Year": active_employees_start_year,
                            "Hires in Year": hires_in_year,
                            "Active Employees End of Year": active_employees_end_year}

# Convert dictionary to DataFrame
df_summary = pd.DataFrame.from_dict(yearly_summary, orient='index')

# Print DataFrame
print(df_summary)


      Terminations in Year  Total Workers in Year  Attrition Rate (%)  \
2013                     8                    256            3.125000   
2014                    17                    278            6.115108   
2015                     5                    289            1.730104   
2016                    15                    316            4.746835   
2017                     9                    312            2.884615   
2018                    13                    325            4.000000   
2019                    10                    346            2.890173   
2020                    11                    368            2.989130   
2021                    25                    372            6.720430   
2022                    28                    370            7.567568   
2023                    14                    353            3.966006   

      Active Employees Start of Year  Hires in Year  \
2013                             149             20   
2014         

In [38]:
import pandas as pd

# Load the data
df_active = pd.read_excel('/Users/timothybrown/Desktop/Active.xlsx')
df_terminations = pd.read_excel('/Users/timothybrown/Desktop/Terminations.xlsx')

# Convert date columns to datetime format
df_active['Adj Service Dt'] = pd.to_datetime(df_active['Adj Service Dt'])
df_terminations['Term Date'] = pd.to_datetime(df_terminations['Term Date'])

# Set year range
years = range(2013, 2024)

# Initialize results storage
results = {}

# Calculate attrition rate for each year
for year in years:
    start_year = f"{year}-01-01"
    end_year = f"{year}-12-31"
    after_year = f"{year+1}-01-01"
    
    # Calculate total workers at the start of year
    active_start_year = df_active[df_active['Adj Service Dt'] < start_year]
    
    # Calculate separations during the year
    separations_year = df_terminations[(df_terminations['Term Date'] >= start_year) & 
                                       (df_terminations['Term Date'] <= end_year)]
    
    # Calculate workers who separated after the year but were active during the year
    active_end_year = df_terminations[(df_terminations['Term Date'] >= after_year) & 
                                      (df_terminations['Hire Date'] <= end_year)]
    
    # Calculate total workers
    total_workers = len(active_start_year) + len(separations_year) + len(active_end_year)
    
    # Calculate attrition rate
    attrition_rate = len(separations_year) / total_workers * 100
    
    # Save results
    results[year] = {"Total Workers": total_workers,
                     "Separations": len(separations_year),
                     "Attrition Rate (%)": attrition_rate}

# Convert dictionary to DataFrame
df_results = pd.DataFrame.from_dict(results, orient='index')

# Print DataFrame
print(df_results)


print(f"Number of active employees at the end of 2023: {len(active_end_2023)}")


      Total Workers  Separations  Attrition Rate (%)
2013            256            8            3.125000
2014            278           17            6.115108
2015            289            5            1.730104
2016            316           15            4.746835
2017            312            9            2.884615
2018            325           13            4.000000
2019            346           10            2.890173
2020            368           11            2.989130
2021            372           25            6.720430
2022            370           28            7.567568
2023            353           14            3.966006
Number of active employees at the end of 2023: 336


In [36]:
average_age_at_termination = df_terminations['Age'].mean()
median_age_at_termination = df_terminations['Age'].median()

print(f"Average age at termination: {average_age_at_termination}")
print(f"Median age at termination: {median_age_at_termination}")


Average age at termination: 48.58064516129032
Median age at termination: 50.0
